In [104]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import json
import re

from sklearn.ensemble import IsolationForest
from tqdm.notebook import tqdm


###########################################
#               FUNCTIONS                 #
###########################################


def clean_reference(ref,outliers):
    for i in outliers:
        ref = ref.drop(labels=i,axis=1)

    return ref

def norm_ref(ref):
    med = ref.median(axis=0)
    norm_ref = ref/med
    return norm_ref, med

def create_synthetic(norm_ref,med,N):
    synt = pd.DataFrame(index=norm_ref.index,columns=range(N))
    for j in range(N):
        for i in norm_ref.index:
            synt[j][i] = np.random.choice(norm_ref.loc[norm_ref.index==i].values.flatten())

        synt[j] = synt[j] * np.random.choice(med)
    synt.columns = ["sample_" + str(i) for i in range(synt.shape[1])]
    return synt

def add_features(synt,sample,gene,factor,exon=None):
    if exon is None:
        pattern = gene
        tmp = [str(synt.index[i]).split("_")[0] for i in range(synt.shape[0])]
        synt.loc[[tmp[i]==pattern for i in range(len(tmp))],sample] = synt.loc[[tmp[i]==pattern for i in range(len(tmp))]][sample] * factor
        
    if exon is not None:
        pattern = gene + "_" + exon
        tmp = [str(synt.index[i]).split("_")[0] + "_" + str(synt.index[i]).split("_")[1] for i in range(synt.shape[0])] 
        synt.loc[[tmp[i]==pattern for i in range(len(tmp))],sample] = synt.loc[[tmp[i]==pattern for i in range(len(tmp))]][sample] * factor
    
    res = synt
    return res


def openJson(path,n):
    """
    Opens json files in path to create a reads matrix
    """
    tmp = os.listdir(path)
    tmp = np.array(tmp)[np.array([bool(re.findall("depths.json$",tmp[i])) for i in range(len(tmp))])]
    reads = np.zeros((n,len(tmp)))
    amplicons = ["" for x in range(n)]
    q=0
    for p in tmp:
        with open(path+p) as json_file:
            data = json.load(json_file)
            for i in range(n):
                amplicons[i] = data[i]['name']
                for j in data[i]['depths']:
                    reads[i,q] = int(data[i]['depths'][j]['min'])
        q=q+1
    reads = pd.DataFrame(data = reads,index=amplicons)
    reads.columns = [i.split('_')[0]+'_'+i.split('_')[1] for i in tmp]
    return reads

def sumLibraries(reads):
    samples = np.unique([i.split('_')[0] for i in reads.columns])
    reads_f = np.zeros((reads.shape[0],len(samples)))
    q=0
    for i in samples:
        sub = reads.filter(regex="^"+i)
        reads_f[:,q] = sub.sum(axis=1)
        q=q+1
    reads_f = pd.DataFrame(data = reads_f,index=reads.index)
    reads_f.columns = list(samples)
    return(reads_f)

def correctIndex(reads,correspondance):
    l = ["" for x in range(len(reads.index))]
    q=0
    for i in reads.index:
        l[q] = i[(len(i)-9):len(i)]
        q=q+1
    final = reads[[correspondance["amplicon"][0] in l[x] for x in range(len(l))]]
    for i in correspondance["amplicon"]:
        if i!=correspondance["amplicon"][0]:
            final = pd.concat([final,reads[[i in l[x] for x in range(len(l))]]])
    final.index = correspondance["gene_exon"] + "_" + correspondance["amplicon"]
    return final


def filterReads(reads,N,output_path):
    reads = reads.loc[:,reads.sum(axis=0)>N]
    reads = reads.filter(regex="^(?!MSI)",axis=0)
    reads = reads.filter(regex="^(?!TN)")
    reads = reads.filter(regex="^(?!TP)")
    reads = reads.filter(regex="^(?!HD)")
    reads = reads.filter(regex="^(?!H2)")
    reads.to_csv(output_path, sep="\t",index=True)
    return(reads)


def normalizeReads(reads,output_path=None,save=False):
    reads_norm=reads/reads.median(axis=0)
    reads = np.log(reads+1)
    if save==True:
        reads_norm.to_csv(output_path, sep="\t",index=True)
    return(reads_norm)


def aberrantSamples(reads,conta='auto'):
    #reads = reads/np.sum(reads)
    
    tmp = np.percentile(reads, 99, axis = 0)/np.mean(reads, axis = 0)
    random_data = np.array(tmp).reshape(-1,1)
    clf = IsolationForest(contamination=conta).fit(random_data)
    preds = clf.predict(random_data)
    res_amp = np.array(reads.columns)[preds==-1]
    
    tmp = np.percentile(reads, 1, axis = 0)/np.mean(reads, axis = 0)
    random_data = np.array(tmp).reshape(-1,1)
    clf = IsolationForest(contamination=conta).fit(random_data)
    preds = clf.predict(random_data)
    res_del = np.array(reads.columns)[preds==-1]
    
    res = np.unique(np.concatenate((res_amp,res_del)))
    norm = reads.columns[~np.in1d(reads.columns,res)]
    
    return(res, norm)


def aberrantSamples2(reads):
    read = reads.astype("float")
    tmp = np.percentile(reads, 5, axis = 0)/np.mean(reads, axis = 0)
    random_data = np.array(tmp).reshape(-1,1)

    clf = IsolationForest(contamination=0.1).fit(random_data)
    preds = clf.predict(random_data)
    res = np.array(reads.columns)[preds==-1]
    return(res)


def aberrantAmplicons(reads_norm,abSamples):
    for name in res:
        random_data = np.array(reads_norm[name]).reshape(-1,1)
        clf = IsolationForest(contamination=0.001).fit(np.array(np.mean(reads_norm, axis = 1)).reshape(-1,1))
        preds = clf.predict(random_data)
        print(name)
        print(np.array(reads_norm.index)[preds==-1])

def aberrantAmpliconsPerSample(name,reads_norm,conta='auto',verbose=False):
    random_data = np.array(reads_norm[name]).reshape(-1,1)
    clf = IsolationForest(contamination=conta).fit(np.array(np.mean(reads_norm, axis = 1)).reshape(-1,1))
    preds = clf.predict(random_data)
    if verbose:
        print(name)
        print(np.array(reads_norm.index)[preds==-1])
    return(np.array(reads_norm.index)[preds==-1])


def aberrantAmpliconsPerSample2(name,reads,abSamples,verbose=False):
    ab = [i in abSamples for i in reads.columns]
    normalReads = reads[np.delete(reads.columns,ab)]
    med = np.percentile(normalReads, 99, axis = 1)
    reads = (reads.T/med).T
    random_data = np.array(reads[name]).reshape(-1,1)
    clf = IsolationForest(contamination=0.05).fit(np.array(np.median(reads, axis = 1)).reshape(-1,1))
    preds = clf.predict(random_data)
    if verbose:
        print(name)
        print(np.array(reads.index)[preds==-1])
    return(np.array(reads.index)[preds==-1])



def percentagePerExon(amplified,reads,verbose=False):
    genes = [i.split('_')[0] for i in reads.index]
    exons = [i.split('_')[1] for i in reads.index]
    g_e = [genes[i]+'_'+exons[i] for i in range(len(genes))]
    n_ge = np.array([g_e.count(i) for i in np.unique(g_e)])
    ag = [i.split('_')[0] for i in amplified]
    ae = [i.split('_')[1] for i in amplified]
    age = [ag[i]+'_'+ae[i] for i in range(len(amplified))]
    f = pd.DataFrame(index=np.unique(age),columns=["percentage"])
    f = f.fillna(0)
    for i in range(len(np.unique(age))):
        f['percentage'][i] = 100*float(age.count(''.join(np.unique(age)[i]))/n_ge[np.unique(g_e)==''.join(np.unique(age)[i])])
        if verbose:
            if f['percentage'][i]>50:
                print(np.unique(age)[i] + ": " + str(round(f['percentage'][i]))+'%'+' des amplicons de l\'exon sont aberrants')
    return(f)

def percentagePerGene(amplified,reads,verbose=False):
    genes = [i.split('_')[0] for i in reads.index]
    ag = [i.split('_')[0] for i in amplified]
    n_g = np.array([genes.count(i) for i in np.unique(genes)])
    f = pd.DataFrame(index=np.unique(ag),columns=["percentage"])
    f = f.fillna(0)
    for i in range(len(np.unique(ag))):
        f['percentage'][i] = 100*float(ag.count(''.join(np.unique(ag)[i]))/n_g[np.unique(genes)==''.join(np.unique(ag)[i])])
        if verbose:
            if f['percentage'][i]>50:
                print(np.unique(ag)[i] + ": " + str(round(f['percentage'][i]))+'%'+' des amplicons du gene sont aberrants')
    return(f)

def amplifEvalGene(reads,abSamples,gene,sample):
    reads_m = reads/reads.median(axis=0)
    sub = reads_m
    for i in abSamples:
        sub = sub.drop(labels=i,axis=1)
    reads_m = reads_m.filter(regex="^"+gene,axis=0)
    reads_m = reads_m[sample]   
    val = np.mean(reads_m)/np.mean(sub.mean())
    if val==np.inf:
        val = 100
    return val

def scoreAmplif(k,n,N):
    p = n/N
    x = np.log(1/((p**k)*(1-p)**(n-k)))*(k/n)
    # score = 1/(1+np.exp(-x))
    score = x/390 + 190/390
    
    return x

def aberrantAmpliconsFinal(reads, reads_norm, abSamples,abSamples2,run,threshold):
    f = pd.DataFrame(columns=["run","name","gene","amplif","score"])
        
    q=0 
    for name in abSamples2:
        #abAmp = aberrantAmpliconsPerSample2(name,reads_norm,abSamples,verbose=False)
        abAmp = aberrantAmpliconsPerSample(name,reads_norm,verbose=False)
        if abAmp.shape!=(0,):
            genes = np.unique([i.split('_')[0] for i in abAmp])
            for gene in genes:
                r = re.compile(gene)
                abEx = list(filter(r.match, abAmp))
                exons1 = [i.split('_')[0]+"_"+i.split('_')[1] for i in abEx]
                tmp = reads.filter(regex="^"+gene,axis=0)
                exons2 = [i.split('_')[0]+"_"+i.split('_')[1] for i in tmp.index]
                
                score = scoreAmplif(len(abEx),tmp.shape[0],reads.shape[0])
                
                amplif = amplifEvalGene(reads, abSamples, gene, name)

                if score>threshold:
                    if amplif>1:
                        f.loc[q] = [run,name,gene,amplif,score]
                        q=q+1
                    #if amplif<1:
                    #    f.loc[q] = [run,name,gene,amplif,score]
                    #    q=q+1

    return(f)


def aberrantAmpliconsFinal2(reads, reads_norm, abSamples,abSamples2,run,threshold):
    f = pd.DataFrame(columns=["run","name","gene","amplif","score"])
        
    q=0 
    for name in abSamples2:
        #abAmp = aberrantAmpliconsPerSample2(name,reads_norm,abSamples,verbose=False)
        abAmp = aberrantAmpliconsPerSample(name,reads_norm,verbose=False)
        if abAmp.shape!=(0,):
            genes = abAmp
            for gene in genes:
                r = re.compile(gene)
                abEx = list(filter(r.match, abAmp))
                #print(abEx)
                tmp = reads.filter(regex="^"+gene,axis=0)                
                score = scoreAmplif(len(abEx),tmp.shape[0],reads.shape[0])
                
                amplif = amplifEvalGene(reads, abSamples, gene, name)

                if score>threshold:
                    if amplif>1:
                        f.loc[q] = [run,name,gene,amplif,score]
                        q=q+1
                    #if amplif<1:
                    #    f.loc[q] = [run,name,gene,amplif,score]
                    #    q=q+1

    return(f)



def aberrantTargetsCapture(abSamples, normalSamples, reads_norm, conta=None, lower=-0.5, upper=0.5, verbose=True, output_path=False):
    if conta == None:
        conta = 1/reads_norm.shape[1]
    f = pd.DataFrame(columns=["name","loc","amp"])
    
    norm = np.mean(reads_norm[normalSamples], axis = 1)

    q=0
    for i in tqdm(reads_norm.columns):            
        x = np.array(reads_norm[i]/norm)

        if sum(reads_norm[i]/norm>1.5)>1:
            #print('duplication: '+i+' '+str(sum(reads_norm[i]/norm>1.4)))
            gg=1
        if sum(reads_norm[i]/norm<0.6)>1:
            #print('deletion:'+i+' '+str(sum(reads_norm[i]/norm<0.6)))
            gg=1        
        
        
        if gg==1:
            random_data = x.reshape(-1, 1)
            clf = IsolationForest(contamination=conta).fit(random_data)
            preds = clf.predict(random_data)

            det = np.array(reads_norm.index)[preds==-1]

            for j in det:
                amp = float(x[reads_norm.index==j])
                if amp<lower:
                    f.loc[q] = [i,j,amp]
                    q=q+1
                if amp>upper:
                    f.loc[q] = [i,j,amp]
                    q=q+1
        gg=0
    
    #for i in tqdm(reads_norm.index):
#
    #    x = np.array(np.log2(reads_norm.loc[i]/float(norm[reads_norm.index==i])))
    #    #pd.DataFrame(x).to_csv("/Users/admin/Documents/CNV/testttt.tsv",sep="\t")
    #    random_data = x.reshape(-1, 1)
    #    clf = IsolationForest(contamination=conta).fit(random_data)
    #    preds = clf.predict(random_data)
#
    #    det = np.array(final_norm.columns)[preds==-1]
    #    ndet = np.array(final_norm.columns)[preds==1]
#
    #    for j in det:
    #        amp = x[final_norm.columns==j]
    #        if amp<lower:
    #            f.loc[q] = [j,i,amp]
    #            q=q+1
    #        if amp>upper:
    #            f.loc[q] = [j,i,amp]
    #            q=q+1
                
    if verbose:
        print(str(f.shape[0])+" aberrant targets detected in "+str(len(np.unique(f['name'])))+" samples")
    return f


In [3]:
pd.set_option('display.max_rows', 2000)
output_path = "/Users/admin/Documents/CNV/"
reads = pd.read_csv("/Users/admin/Documents/CNV/final_ICR96_good.txt",sep="\t",index_col=0)

abSamples, normalSamples = aberrantSamples(reads,conta=0.5)
#pd.DataFrame(normalSamples).to_csv("/Users/admin/Documents/CNV/normal_samples_icr.tsv",sep="\t")

final_norm = normalizeReads(reads,output_path,save=False)

f = aberrantTargetsCapture(abSamples, normalSamples, final_norm,conta='auto',upper=0,lower=0)

f.to_csv("/Users/admin/Documents/CNV/icr_res/res_capture_ICR96_test.txt", sep="\t",index=False)



16234 aberrant targets detected in 69 samples


In [ ]:
from tqdm.notebook import tqdm

tp=[]
fp=[]
tn=[]
fn=[]
for k in tqdm(range(5,101,5)):
    factor = k/10
    for p in (range(1000)):
        synt = pd.read_csv("/Users/admin/Documents/CNV/synthetic_raw_85.tsv",sep="\t",index_col=0)
        synt = synt[np.random.choice(synt.columns,30,replace=False)]
        col = np.random.choice(synt.columns,size=1,replace=False)
        normalSamples = synt.columns[~np.in1d(synt.columns,col)]

        row = np.random.choice(synt.index,size=1)
        #row = pd.read_csv("/Users/admin/Documents/CNV/article/figs/data_fig3/rows.tsv",sep="\t")
        #row = np.array(row)

        neg = synt.index[~np.in1d(synt.index,row)]
        pos = row

        for i in row:
            synt.at[i,col[0]] = synt.at[i,col[0]]*factor

        synt_norm = normalizeReads(synt)

        conta = 'auto'#1/synt_norm.shape[0]

        norm = np.mean(synt_norm[normalSamples], axis = 1)

        x = np.array(synt_norm[col]/norm[0])     

        random_data = x.reshape(-1, 1)
        clf = IsolationForest(contamination=conta).fit(random_data)
        preds = clf.predict(random_data)

        det = np.array(synt_norm.index)[preds==-1]
        ndet = np.array(synt_norm.index)[preds==1]


        tp.extend([sum(np.in1d(det,pos))])
        fp.extend([sum(np.in1d(det,neg))])
        tn.extend([sum(np.in1d(ndet,neg))])
        fn.extend([sum(np.in1d(ndet,pos))])

pd.DataFrame(tp).to_csv("/Users/admin/Documents/CNV/article/figs/data_fig3/tp_amp.tsv", sep="\t",index=False)
pd.DataFrame(fp).to_csv("/Users/admin/Documents/CNV/article/figs/data_fig3/fp_amp.tsv", sep="\t",index=False)
pd.DataFrame(fn).to_csv("/Users/admin/Documents/CNV/article/figs/data_fig3/fn_amp.tsv", sep="\t",index=False)
pd.DataFrame(tn).to_csv("/Users/admin/Documents/CNV/article/figs/data_fig3/tn_amp.tsv", sep="\t",index=False)


In [255]:
synt_norm.shape[1]

30

In [42]:

def aberrantTargetsCapture(abSamples, normalSamples, reads_norm, conta=None, lower=-0.5, upper=0.5, verbose=True, output_path=False):
    if conta == None:
        conta = 1/reads_norm.shape[1]
    f = pd.DataFrame(columns=["name","loc","amp"])
    
    norm = np.mean(reads_norm[normalSamples], axis = 1)
    reads_norm_norm = reads_norm/norm
    
    
    q=0
    for i in tqdm(reads_norm.columns):
        x = reads_norm[i]/norm
        x = np.array(np.log2(x[reads_norm[i]!=0]))
        #pd.DataFrame(x).to_csv("/Users/admin/Documents/CNV/icr_res/"+i+".tsv",sep="\t")
        random_data = x.reshape(-1, 1)
        clf = IsolationForest(contamination=conta).fit(random_data)
        preds = clf.predict(random_data)

        det = np.array(final_norm.index)[reads_norm[i]!=0][preds==-1]
        ndet = np.array(final_norm.index)[reads_norm[i]!=0][preds==1]

        for j in det:
            amp = np.log2(reads_norm[i][j]/norm[j])
            f.loc[q] = [i,j,amp]
            if amp<lower:
                f.loc[q] = [i,j,amp]
                q=q+1
            if amp>upper:
                f.loc[q] = [i,j,amp]
                q=q+1
                
    if verbose:
        print(str(f.shape[0])+" aberrant targets detected in "+str(len(np.unique(f['name'])))+" samples")
    return f

array(['17296', '17297', '17298', '17299', '17300', '17301', '17302',
       '17303', '17304', '17307', '17321', '17323', '17325', '17330',
       '17333', '17334', '17335', '17339', '17356', '17358', '17362',
       '17363', '17366', '17373', '17377', '17381', '17383', '17384',
       '17388', '17392', '17393', '17394', '17396'], dtype=object)